In [1]:
from heeps.util.multiCPU import multiCPU
from heeps.util.freq_decomp import fit_zer, remove_zernike
import numpy as np
import os  
from astropy.io import fits
from copy import deepcopy
import proper

In [2]:
os.chdir(os.path.normpath(os.path.expandvars('$HOME/heeps_metis/input_files/wavefront')))
scao = fits.getdata('cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_scao_only_L_285.fits')
cbw = fits.getdata('ncpa/LM_20211122_L_285_rep_15.fits')
wv = fits.getdata('wv/cube_Cbasic_20210601_3600s_300ms_0piston_meters_scao_only_L_285_WVonly.fits')
ncpa_name = 'cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_L_285.fits'
ncpa = scao + cbw + wv

In [3]:
pup = fits.getdata('cfull/mask_Cfull_20220512_L_285.fits')
pup[pup < .5] = 0
nimg = 285
nzer = 100
freq = 3
G = 0.4
lag = 3
sigLF = 13e-9
sigHF = 3e-9

In [4]:
zpols_name = ncpa_name[:-5] + '_zpols_%s.fits'%nzer
try:
    zpols = fits.getdata(zpols_name)
    print('getdata ' + zpols_name)
except FileNotFoundError:
    print('writeto ' + zpols_name)
    zpols = multiCPU(fit_zer, posargs=[pup, nimg/2, nzer], 
        posvars=[ncpa], case='get zpols')
    fits.writeto(zpols_name, np.float32(zpols))

getdata cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_L_285_zpols_100.fits


In [5]:
zpols_integ_name = ncpa_name[:-5] + '_zpols_%s_freq_%s_G_%s_lag_%s_sigLF_%s_sigHF_%s.fits'%(nzer, freq, G, lag, sigLF, sigHF)
if os.path.isfile(zpols_integ_name):
    zpols_integ = fits.getdata(zpols_integ_name)
    print('getdata ' + zpols_integ_name)
else:
    print('write to ' + zpols_integ_name)
    zpols_integ = np.zeros(zpols.shape)
    nframes = len(zpols)
    # piston
    zpols_integ[:,0] = zpols[:,0]
    # tip-tilt, and higher modes
    for m, sig in zip([range(1,3), range(3,nzer)], [sigLF, sigHF]):
        for n in range(freq+lag, nframes, freq):
            error = np.mean(zpols[n-freq-lag:n-lag,m] - zpols_integ[n-1,m] + np.random.normal(0, sig, (freq,len(m))), 0) 
            zpols_integ[n:n+freq,m] = zpols_integ[n-1,m] + G*error
    fits.writeto(zpols_integ_name, np.float32(zpols_integ))

write to cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_L_285_zpols_100_freq_3_G_0.4_lag_3_sigLF_1.3e-08_sigHF_3e-09.fits


In [6]:
if os.path.isfile(ncpa_name):
    HSF = fits.getdata(ncpa_name)
    print('getdata ' + ncpa_name)
else:
    print('write to ' + ncpa_name)
    wf = proper.prop_begin(1, 1, nimg, 1) # initial wavefront
    _, HSF = multiCPU(remove_zernike, multi_out=True, verbose=True,
        posargs=[deepcopy(wf), pup],
        posvars=[ncpa, zpols_integ])
    fits.writeto(ncpa_name, np.float32(HSF))

write to cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_all_ncpa_L_285.fits
   2022-06-22 19:26:47, using 56 cores
   2022-06-22 19:43:33, completed in 1006.29 seconds


In [7]:
print('SCAO+NCPA: %.2f nm'%(np.mean([np.std(x[pup!=0]) for x in HSF*1e9])))

SCAO+NCPA: 125.53 nm
